### Model Trainer

In [1]:
import os
%pwd

'/Users/sumi/Desktop/summarizer/research'

In [2]:
os.chdir("../")
%pwd

'/Users/sumi/Desktop/summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [4]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(PARAMS_FILE_PATH)
        
        create_directories([self.config.artifacts_root])
       
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config=ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path= config.data_path,
            model_ckpt= config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps   
        )
        
        return model_trainer_config
         

In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
import torch
from datasets import load_from_disk

/Users/sumi/Desktop/summarizer/textenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-11-19 19:36:30,245: INFO: config: PyTorch version 2.5.1 available.]


In [7]:
class ModelTrainer:
    def __init__(self, config:ModelTrainerConfig):
        self.config=config
        
    def train(self):
        device = device = torch.device("cpu")  #"mps" if torch.backends.mps.is_available() else "cpu"
        tokenizer=AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        # Loading the data
        dataset_samsum_pt=load_from_disk(self.config.data_path)
        
        trainer_args=TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy="steps", eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )
        
        trainer=Trainer(model=model_pegasus, args=trainer_args,
                    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                    train_dataset=dataset_samsum_pt["test"], # Just to make sure training happens quickly
                    eval_dataset=dataset_samsum_pt["validation"])

        trainer.train()
        
        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        
        
        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        
        
        
        

In [8]:
import os
os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"


In [9]:
config=ConfigurationManager()
model_trainer_config=config.get_model_trainer_config()
model_trainer_config=ModelTrainer(config=model_trainer_config)
model_trainer_config.train()

[2024-11-19 19:36:30,443: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-19 19:36:30,445: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-19 19:36:30,445: INFO: common: created directory at: artifacts]
[2024-11-19 19:36:30,445: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/sumi/Desktop/summarizer/textenv/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/vm/62zyvxyx70qb5rrghd3vfnx80000gn/T/ipykernel_5109/1489979815.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer=Trainer(model=model_pegasus, args=trainer_args,
  2%|▏         | 1/51 [09:19<7:46:36, 559.94s/it]

KeyboardInterrupt: 

In [ ]:
# Run in colab and download the files inside the 
# pegasus-samsum-model and tokenize save in artifacts/model_trainer also tehcheckpoint needs to be downloaded